In [10]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

import warnings
warnings.filterwarnings("ignore")

maup.progress.enabled = True

In [11]:
# parameters
# state = Georgia
state_ab = "ga"

# Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [60]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2021/CONGRESS-PROP1-2021-shape.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2021/SENATE-PROP1-2021-Shape.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2021/HOUSE-PROP1-2021-shape.shp".format(data_folder, state_ab)

In [13]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [14]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [15]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [16]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [17]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [18]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [19]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [20]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [21]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['P0020005'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['P0020006'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['P0020007'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['P0020008'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['P0020009'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['P0020010'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['P0020011'], 1)

In [22]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
                    'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
                    'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
                                        'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [23]:
population_df.rename(columns=rename_dict, inplace = True)

### Read the vest 16 data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [24]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

In [25]:
vest16 = gpd.read_file(vest16_data)

In [26]:
if maup.doctor(vest16) != True:
    vest16 = do_smart_repair(vest16)

100%|████████████████████████████████████████| 2697/2697 [00:11<00:00, 238.07it/s]


There are 294 overlaps.
There are 2029 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 4665/4665 [00:03<00:00, 1503.66it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|████████████████████████████████████████| 2697/2697 [00:10<00:00, 247.92it/s]


In [32]:
maup.doctor(vest16)

100%|████████████████████████████████████████| 2697/2697 [00:12<00:00, 223.77it/s]


There are some invalid geometries.


False

In [41]:
# I tried to run maup.assign and failed -> TopologyException: Input geom 1 is invalid: Self-intersection at or near point

In [33]:
vest16 = do_smart_repair(vest16)

Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 3119/3119 [00:02<00:00, 1318.78it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|████████████████████████████████████████| 2697/2697 [00:10<00:00, 248.45it/s]


In [34]:
maup.doctor(vest16)

100%|████████████████████████████████████████| 2697/2697 [00:10<00:00, 248.52it/s]


True

In [27]:
vest16.columns

Index(['DISTRICT', 'CTYSOSID', 'PRECINCT_I', 'PRECINCT_N', 'CTYNAME',
       'CTYNUMBER', 'CTYNUMBER2', 'FIPS2', 'G16PRERTRU', 'G16PREDCLI',
       'G16PRELJOH', 'G16USSRISA', 'G16USSDBAR', 'G16USSLBUC', 'G16PSCRECH',
       'G16PSCLHOS', 'geometry'],
      dtype='object')

In [35]:
start_col = 8 # this should be the same for all vest data in that state
year = '16'
vest_base_data = vest16

In [36]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [37]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|█████████████████████████████████████████| 2697/2697 [01:43<00:00, 26.03it/s]


In [38]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 
                    'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 
           'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [39]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [40]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

## Check if population agrees

In [42]:
check_population(population_df, vest_base)

population agrees


In [43]:
vest18 = gpd.read_file(vest18_data)

In [44]:
vest18.columns

Index(['DISTRICT', 'CTYSOSID', 'PRECINCT_I', 'PRECINCT_N', 'CTYNAME',
       'CTYNUMBER', 'CTYNUMBER2', 'FIPS2', 'G18GOVRKEM', 'G18GOVDABR',
       'G18GOVLMET', 'G18LTGRDUN', 'G18LTGDAMI', 'G18SOSRRAF', 'G18SOSDBAR',
       'G18SOSLDUV', 'G18ATGRCAR', 'G18ATGDBAI', 'G18AGRRBLA', 'G18AGRDSWA',
       'G18INSRBEC', 'G18INSDLAW', 'G18INSLFOS', 'G18SPIRWOO', 'G18SPIDTHO',
       'G18LABRBUT', 'G18LABDKEA', 'G18PSCREAT', 'G18PSCDMIL', 'G18PSCLGRA',
       'G18PSCRPRI', 'G18PSCDRAN', 'G18PSCLTUR', 'R18SOSRRAF', 'R18SOSDBAR',
       'R18PSCREAT', 'R18PSCDMIL', 'geometry'],
      dtype='object')

In [54]:
# election_df = add_vest(vest18, election_df, '18', population_df, start_col)
# First time running add_vest and failed on maup.assign (not smart repair, so here I ran the smart repair once)

In [47]:
maup.doctor(vest18)

100%|████████████████████████████████████████| 2658/2658 [00:10<00:00, 242.11it/s]


There are 306 overlaps.
There are 1001 holes.


False

In [48]:
vest18 = do_smart_repair(vest18) # second time running smart_repair

Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 4653/4653 [00:02<00:00, 1579.42it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|████████████████████████████████████████| 2658/2658 [00:10<00:00, 251.02it/s]


In [49]:
maup.doctor(vest18)

100%|████████████████████████████████████████| 2658/2658 [00:10<00:00, 253.89it/s]


There are some invalid geometries.


False

In [50]:
vest18 = do_smart_repair(vest18) # third time running smart_repair

Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 3089/3089 [00:02<00:00, 1298.01it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]


100%|████████████████████████████████████████| 2658/2658 [00:10<00:00, 251.47it/s]


In [51]:
maup.doctor(vest18)

100%|████████████████████████████████████████| 2658/2658 [00:10<00:00, 252.10it/s]


True

In [52]:
election_df = add_vest(vest18, election_df, '18', population_df, start_col)

100%|█████████████████████████████████████████| 2697/2697 [01:43<00:00, 26.14it/s]


population agrees


In [33]:
vest20 = gpd.read_file(vest20_data)

In [34]:
vest20.columns

Index(['DISTRICT', 'CTYSOSID', 'PRECINCT_I', 'PRECINCT_N', 'CTYNAME',
       'CTYNUMBER', 'CTYNUMBER2', 'FIPS2', 'G20PRERTRU', 'G20PREDBID',
       'G20PRELJOR', 'C20PRERTRU', 'C20PREDBID', 'C20PRELJOR', 'G20USSRPER',
       'G20USSDOSS', 'G20USSLHAZ', 'S20USSRLOE', 'S20USSRCOL', 'S20USSRGRA',
       'S20USSRJAC', 'S20USSRTAY', 'S20USSRJOH', 'S20USSDWAR', 'S20USSDJAC',
       'S20USSDLIE', 'S20USSDJOH', 'S20USSDJAM', 'S20USSDSLA', 'S20USSDWIN',
       'S20USSDTAR', 'S20USSLSLO', 'S20USSGFOR', 'S20USSIBUC', 'S20USSIBAR',
       'S20USSISTO', 'S20USSIGRE', 'G20PSCRSHA', 'G20PSCDBRY', 'G20PSCLMEL',
       'G20PSCRMCD', 'G20PSCDBLA', 'G20PSCLWIL', 'R21USSRPER', 'R21USSDOSS',
       'R21USSRLOE', 'R21USSDWAR', 'R21PSCRMCD', 'R21PSCDBLA', 'geometry'],
      dtype='object')

## The Vest20 data
Vest20 data does not load, due to topology error.  So it's not in the final json

In [35]:
election_df = add_vest(vest20, election_df, '20', population_df, start_col)

GEOSException: TopologyException: side location conflict at -84.281015999999994 33.772047000000001. This can occur if the input geometry is invalid.

## Add the district data

In [61]:
cong_df = gpd.read_file(cd_data)
send = gpd.read_file(send_data)
hdist = gpd.read_file(hdist_data)

In [62]:
print(cong_df.columns)
print(send.columns)
print(hdist.columns)

Index(['ID', 'AREA', 'DATA', 'DISTRICT', 'MEMBERS', 'LOCKED', 'NAME',
       'POPULATION', 'F18_POP', 'NH_WHT', 'NH_BLK', 'HISPANIC_O', 'NH_ASN',
       'NH_IND', 'NH_HWN', 'NH_OTH', 'NH_2_RACES', 'NH18_WHT', 'NH18_BLK',
       'H18_POP', 'NH18_ASN', 'NH18_IND', 'NH18_HWN', 'NH18_OTH', 'NH18_2_RAC',
       'AP_WHT', 'AP_BLK', 'AP_IND', 'AP_ASN', 'AP_HWN', 'AP_OTH',
       'F18_AP_WHT', 'F18_AP_BLK', 'F18_AP_IND', 'F18_AP_ASN', 'F18_AP_HWN',
       'F18_AP_OTH', 'PEOPLE_IN_', 'COLORING', 'DEVIATION', 'F_DEVIATIO',
       'F_NH_WHT', 'F_AP_BLK', 'F_HISPANIC', 'F_NH18_WHT', 'F_18_AP_BL',
       'F_H18_POP', 'F_18_POP', 'F_NH_BLK', 'F_NH_ASN', 'F_NH_IND', 'F_NH_HWN',
       'F_NH_2_RAC', 'F_NH18_BLK', 'F_NH18_ASN', 'F_NH18_IND', 'F_NH18_HWN',
       'F_NH18_2_R', 'F_NH_OTH', 'F_NH18_OTH', 'F_AP_WHT', 'F_AP_IND',
       'F_AP_ASN', 'F_AP_HWN', 'F_AP_OTH', 'F_18_AP_WH', 'F_18_AP_IN',
       'F_18_AP_AS', 'F_18_AP_HW', 'F_18_AP_OT', 'F_PEOPLE_I', 'IDEAL_VALU',
       'geometry'],
      dtype=

In [63]:
election_df = add_district(cong_df, "CD", election_df, "DISTRICT")

100%|█████████████████████████████████████████████| 14/14 [00:09<00:00,  1.55it/s]


In [64]:
election_df = add_district(send, "SEND", election_df, "DISTRICT")

100%|█████████████████████████████████████████████| 56/56 [00:11<00:00,  4.71it/s]


In [65]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICT")

100%|███████████████████████████████████████████| 180/180 [00:15<00:00, 11.38it/s]


In [69]:
list(election_df.columns)

['2MOREVAP',
 'AGR18D',
 'AGR18R',
 'AMINVAP',
 'ASIANVAP',
 'ATG18D',
 'ATG18R',
 'BVAP',
 'CTYNAME',
 'CTYNUMBER',
 'CTYNUMBER2',
 'CTYSOSID',
 'DISTRICT',
 'FIPS2',
 'GOV18D',
 'GOV18O',
 'GOV18R',
 'HISP',
 'HVAP',
 'H_2MORE',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'INS18D',
 'INS18O',
 'INS18R',
 'LAB18D',
 'LAB18R',
 'LTG18D',
 'LTG18R',
 'NHPIVAP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'OTHERVAP',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRECINCT_I',
 'PRECINCT_N',
 'PSC16O',
 'PSC16R',
 'PSC18D',
 'PSC18O',
 'PSC18R',
 'SOS18D',
 'SOS18O',
 'SOS18R',
 'SPI18D',
 'SPI18R',
 'TOTPOP',
 'USS16D',
 'USS16O',
 'USS16R',
 'VAP',
 'WVAP',
 'geometry',
 'CD',
 'SEND',
 'HDIST']

In [70]:
base_columns = {}
if 'CTYSOSID' + year not in election_df.columns:
    base_columns = {
        'DISTRICT':'DISTRICT'+year,
        'CTYSOSID':'CTYSOSID'+year,
        'PRECINCT_I':'PRECINCT_I'+year,
        'PRECINCT_N':'PRECINCT_N'+year,
        'CTYNAME':'CTYNAME'+year,
        'CTYNUMBER':'CTYNUMBER'+year,
        'CTYNUMBER2':'CTYNUMBER2'+year,
        'FIPS2':'FIPS2'+year,
    }
election_df.rename(columns=base_columns, inplace = True)

In [71]:
# reorder the columns
fixed_columns = [
    'DISTRICT'+year,
    'CTYSOSID'+year,
    'PRECINCT_I'+year,
    'PRECINCT_N'+year,
    'CTYNAME'+year,
    'CTYNUMBER'+year,
    'CTYNUMBER2'+year,
    'FIPS2'+year,
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [74]:
# store the result in directory "{state_abbreviation}"
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))